Implement and apply Lesk’s algorithm to the publicly available data set of SemEval 2013 Shared Task \#12 \(Navigli and Jurgens, 2013\), using NLTK’s interface to WordNet v3.0 as your lexical resource.
(Be sure you are using WordNet v3.0!) The relevant files are available on the course website. Starter
code is also provided to help you load the data. More information on the data set can be found at
https://www.cs.york.ac.uk/semeval-2013/task12/.
The provided code will load all of the cases that you are to resolve, along with their sentential context.
Apply word tokenization and lemmatization (you have code to do this from A1) as necessary, and remove
stop words.
As a first step, compare the following two methods for WSD:

1. The most frequent sense baseline: this is the sense indicated as #1 in the synset according to
WordNet
2. NLTK’s implementation of Lesk’s algorithm (nltk.wsd.lesk)
Use accuracy as the evaluation measure. There is sometimes more than one correct sense annotated in
the key. If that is the case, you may consider an automatic system correct if it resolves the word to any
one of those senses. What do you observe about the results?

# Imports and Environment Setup

In [149]:
%run loader.py

In [150]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/2013/dgarfi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [157]:
nltk.download('stopwords')
lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

def lemmatize_sentence(list_or_str):
    if isinstance(list_or_str, str):
        sentence_list = list_or_str.split()
    else:
        sentence_list = list_or_str
    return [lemmatizer.lemmatize(word) for word in sentence_list if word not in stopwords]

def preprocess_wsdinstance(wsdinstance):
    wsdinstance.context = lemmatize_sentence(wsdinstance.context)
    return wsdinstance
    
dev_instances = { k: preprocess_wsdinstance(wsdinstance) for k, wsdinstance in dev_instances.items() }

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/2013/dgarfi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Calculating Accuracy

In [158]:
def get_synsets_from_keys(keys_dict):
    """
    :param keys_dict: a dictionary of word-key: [synset ids] mappings
    :return dict: returns the same dictionary but maps [synset ids] to their corresponding Synset() objects
    """
    return { k: [wn.lemma_from_key(key).synset() for key in list_of_keys]
               for k, list_of_keys in keys_dict.items()}

expected_dev = get_synsets_from_keys(dev_key)
expected_test = get_synsets_from_keys(test_key)

def accuracy_of(predicted, expected):
    """
    :param predicted dict: word-key: Synset() dictionary of predicted word-sense disambiguations
    :param expected dict: word-key: Synset() of expected word-sense disambiguations    
    :return float: the percent of total keys in expected which are correctly predicted
    
    Note that expected.values() maps our word-based lexicon to lists of Synsets.
    If any one of its possible Synsets is predicted, the word is considered correcty predicted.
    """
    assert(len(predicted.values()) == len(expected.values()))
    return sum(map(lambda x: x[0] in x[1], zip(predicted.values(), expected.values()))) / float(len(predicted.values()))

# Part 1: Baseline and Lesk's Algorithm

In [159]:
def predict_with(method, instance_dict):
    return { k: method(wsd) for k, wsd in instance_dict.items() }
    
def baseline(target_instance):
    return wn.synsets(target_instance.lemma)[0]

predicted_baseline = predict_with(baseline, test_instances)

accuracy_of(predicted_baseline, expected_test)

0.038620689655172416

In [160]:
predicted_lesk = { k: nltk.wsd.lesk(wsd.context, wsd.lemma) for k, wsd in test_instances.items() }

accuracy_of(predicted_lesk, expected_test)

0.022758620689655173

# Part 2: Combination of Word Sense Frequency & Lesk's Algorithm